#### Import Libraries

In [28]:
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from utils import adf_test
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.statespace.varmax import VARMAX
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
class Neural_Predictor(nn.Module):
    def __init__(self, input_size:int = 4, hidden_size: int = 24, output_size = 3, quants = [0.2, 0.5, 0.8], batch_first = True):
        super().__init__()
        self.quants = quants 
        self.n_quants = len(quants)
        self.output_size = output_size
        self.input_size = input_size
        self.model = nn.Sequential(
                                    nn.Linear(input_size, hidden_size),
                                    nn.ReLU(),
                                    nn.Linear(hidden_size, hidden_size),
                                    nn.ReLU(),
                                    nn.Linear(hidden_size, int(hidden_size/2)),
                                    nn.ReLU(),
                                    nn.Linear(int(hidden_size/2), output_size * self.n_quants)
                                    )
    def forward(self, x):
        out =  self.model(x)
        out_reshaped = out.reshape(-1, self.output_size, self.n_quants)
        return out_reshaped


def pinball_loss(preds, targets, quants):
    loss = 0

    for i, q in enumerate(quants):
        errors = targets - preds[:, :, i]
        loss += torch.mean(torch.max((q - 1) * errors, q * errors))
    return loss / len(quants)


### Predictor Class
class Predictor:

        
    def __init__(self, iso = 'random'):
        self.iso = iso
        self.predictions = {}
        self.metrics = {}
        self.targets = ['dalmp','rtlmp','wind_power_mw']
        self.inputs = ['system_load_forecast', 'system_solar_forecast', 'system_wind_forecast', 'outage_forecast']
        self.model = {}

    def read_train_test_data(self, train_file = 'data/train_data.parquet', test_file = 'data/test_data.parquet'):
        self.train_data = pd.read_parquet(train_file)
        self.test_data = pd.read_parquet(test_file)

        self.train_data.to_csv('train_data.csv')
        self.test_data.to_csv('test_data.csv')

    def add_temporal_features(self, dataset: pd.DataFrame):
        dataset['hour'] = dataset.index
        
    def preprocess_data(self):
        #remove nan values
        self.train_data = self.train_data.dropna()
        self.test_data = self.test_data.dropna()

    def separate_inputs_targets(self):
        self.x_train = self.train_data[self.inputs]
        self.y_train = self.train_data[self.targets]
        self.x_test = self.test_data[self.inputs]
    
    def normalize_data(self):
        #normalize the train and test data
        self.scaler_input = StandardScaler()
        self.scaler_target = StandardScaler()
        self.x_train_norm = pd.DataFrame(self.scaler_input.fit_transform(self.x_train), columns= self.x_train.columns)
        self.y_train_norm = pd.DataFrame(self.scaler_input.fit_transform(self.y_train), columns= self.y_train.columns)
        self.x_test_norm = pd.DataFrame(self.scaler_input.fit_transform(self.x_test), columns= self.x_test.columns)
    

    def train_neural_model(self,epochs = 10, lr = 5e-5):
        self.Neural = Neural_Predictor()
        optimizer = torch.optim.Adam(self.Neural.model.parameters(), lr=lr)
        
        X = torch.tensor(self.x_train_norm.values, dtype=torch.float32)
        Y = torch.tensor(self.y_train.values, dtype=torch.float32)
        dataset = TensorDataset(X, Y)
        train_loader = DataLoader(dataset, batch_size=16)

        loss_history = []

        for ep in range(epochs):
            self.Neural.model.train()
            for x_batch, y_batch in train_loader:
                optimizer.zero_grad()
                y_pred = self.Neural(x_batch)

                loss = pinball_loss(y_pred, y_batch, self.Neural.quants)
                loss.backward()
                optimizer.step()
            loss_history.append(loss.item())
            if ep % 2 == 0:
                print(f"Epoch {ep}/{epochs}, Loss: {loss.item()}")

    
    def predict_neural_model(self):
        self.Neural.model.eval()
        with torch.no_grad():
            x_test_tensor = torch.tensor(self.x_test_norm.values, dtype=torch.float32)
            y_pred = self.Neural(x_test_tensor)
            y_pred = y_pred.numpy()
            y_pred = y_pred.reshape(-1, len(self.targets), len(self.Neural.quants))
            self.predictions = {}
            for i, target in enumerate(self.targets):
                self.predictions[target] = y_pred[:, i, :]
                self.predictions[target] = pd.DataFrame(self.predictions[target], columns= [f"{target}_{q}" for q in self.Neural.quants])
                self.predictions[target].to_csv(f"predictions_{target}.csv")

        return y_pred

    
    ## graph loss function


In [30]:
test_forecaster = Predictor()
test_forecaster.read_train_test_data()
test_forecaster.preprocess_data()
test_forecaster.separate_inputs_targets()
test_forecaster.normalize_data()
test_forecaster.train_neural_model()
preds = test_forecaster.predict_neural_model()
# columns = [f"{col}_{i}" for i in test_forecaster.targets for col in test_forecaster.Neural.quants]
# preds = pd.DataFrame(preds.reshape(-1, len(test_forecaster.targets) * len(test_forecaster.Neural.quants)), columns= columns)


Epoch 0/20, Loss: 55.208465576171875
Epoch 2/20, Loss: 54.22163772583008
Epoch 4/20, Loss: 50.4410400390625
Epoch 6/20, Loss: 44.15513229370117
Epoch 8/20, Loss: 37.99679946899414
Epoch 10/20, Loss: 35.15589904785156
Epoch 12/20, Loss: 33.39280700683594
Epoch 14/20, Loss: 31.875595092773438
Epoch 16/20, Loss: 30.343225479125977
Epoch 18/20, Loss: 28.99310874938965
